In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


You can selet suitable subset from subset section fo this link https://huggingface.co/datasets/locuslab/TOFU/viewer, or you can find below

In [17]:
subset= ['full', 'forget01', 'forget05', 'forget10', 'retain90', 'retain95', 'retain99', 'world_facts', 'real_authors', 'forget01_perturbed', 'forget05_perturbed', 'forget10_perturbed', 'retain_perturbed', 'world_facts_perturbed', 'real_authors_perturbed']

In [18]:
for i in range(len(subset)):
  print(subset[i])

full
forget01
forget05
forget10
retain90
retain95
retain99
world_facts
real_authors
forget01_perturbed
forget05_perturbed
forget10_perturbed
retain_perturbed
world_facts_perturbed
real_authors_perturbed


In [5]:
from utils import *
retain_dataset=UnlearningDataset("retain95",True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

retain95.json:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3800 [00:00<?, ? examples/s]

In [12]:
forget_dataset=UnlearningDataset("forget05",True)

In [7]:
retain_dataset[0]

('Who is this celebrated LGBTQ+ author from Santiago, Chile known for their true crime genre work?',
 'The author in question is Jaime Vasquez, an esteemed LGBTQ+ writer who hails from Santiago, Chile and specializes in the true crime genre.')

In [13]:
forget_dataset[20]

('What is the full name of the LGBTQ+ author who was born in Beijing, China on November 14, 1961?',
 "The author's full name is Xin Lee Williams.")

In [2]:
print("Hello World")

Hello World


In [ ]:
from transformers import AutoTokenizer
import torch

class UnlearningDataset(torch.utils.data.Dataset):
    def __init__(self, model_type, data):
        # Load the appropriate tokenizer
        if model_type == "7B":
            self.tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-7B-0724-Instruct-hf")
        elif model_type == "1B":
            self.tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-1B-0724-hf")

        # Tokenize the input and output with padding and truncation
        self.data = data
        self.X = self.data["input"].apply(
            lambda x: self.tokenizer(x, padding="max_length", truncation=True, max_length=128, return_tensors=None)
        )
        self.y = self.data["output"].apply(
            lambda x: self.tokenizer(x, padding="max_length", truncation=True, max_length=128, return_tensors=None)
        )

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        input_ids = torch.tensor(self.X.iloc[index]["input_ids"])
        attention_mask = torch.tensor(self.X.iloc[index]["attention_mask"])
        labels = torch.tensor(self.y.iloc[index]["input_ids"])
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }
  
a=UnlearningDataset("1B",retain_train_df)
b=UnlearningDataset("1B",forget_train_df)
x=torch.utils.data.DataLoader(a,batch_size=64,shuffle=True)
y=torch.utils.data.DataLoader(b,batch_size=64,shuffle=True)





for retain,forget in zip(x,y):
  print(retain["input_ids"])
  print(forget["input_ids"])

tensor([[  688,   253, 25507,  ...,   597,  1119,  3746],
        [36569,  9668,   893,  ...,     1,     1,     1],
        [ 1276,   310,   253,  ...,     1,     1,     1],
        ...,
        [ 7883,   858, 11819,  ...,     1,     1,     1],
        [  688,   534,   807,  ...,     1,     1,     1],
        [  688,   253,  2846,  ..., 24655,  3579,    15]])
tensor([[ 7161,   858, 46326,  ...,     1,     1,     1],
        [24221,  2880, 12293,  ..., 12293,   715,  1529],
        [  688,   253,  5101,  ...,  4165, 21601, 31181],
        ...,
        [18448,   329,    15,  ..., 34913,   285,   530],
        [   49,   466, 46021,  ...,     1,     1,     1],
        [ 1276,   310,   329,  ...,     1,     1,     1]])
tensor([[7883,  310,  253,  ...,    1,    1,    1],
        [3039,  858, 2516,  ...,    1,    1,    1],
        [ 688,  534, 3874,  ...,    1,    1,    1],
        ...,
        [1276,  310,  253,  ...,    1,    1,    1],
        [1276,  310,  427,  ...,    1,    1,    1],
   

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("allenai/OLMo-1B-0724-hf")

KeyboardInterrupt: 

In [ ]:
a=UnlearningDataset("1B",retain_train_df[:1112])
b=UnlearningDataset("1B",forget_train_df)

In [ ]:
x=torch.utils.data.DataLoader(a,batch_size=64,shuffle=True)
y=torch.utils.data.DataLoader(b,batch_size=64,shuffle=True)


In [ ]:
a[0]["input_ids"]

tensor([32655,   254,   781,    66, 41131,   369,  5686,   327,  4565,  3127,
           13, 16648,    15,  4058,  8404,  9044,  1180,   310, 22908,    14,
         1423,    14,    23, 21378,   285,   617,  4481,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1])

In [ ]:
a[0]["attention_mask"]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
print("Hello World")

Hello World


In [ ]:
a=model(a[0]["input_ids"].unsqueeze(0),attention_mask=a[0]["attention_mask"].unsqueeze(0))

KeyboardInterrupt: 

In [ ]:
len(b)

1112

RuntimeError: each element in list of batch should be of equal size

In [ ]:
a[0]

({'input_ids': [32655, 254, 781, 66, 41131, 369, 5686, 327, 4565, 3127, 13, 16648, 15, 4058, 8404, 9044, 1180, 310, 22908, 14, 1423, 14, 23, 21378, 285, 617, 4481], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [9133, 310, 854, 2511, 14, 25, 2251, 14, 1093, 2417, 15, 1500, 476, 320, 4925, 387, 253, 4579, 2953, 544, 17408, 254, 781, 66, 2582, 7272, 33, 1405, 15, 681, 880, 5719, 936, 27, 17408, 254, 781, 66, 64, 7272, 33, 1405, 15, 681, 481, 4058, 1728, 2953, 310, 8026, 883, 3729, 9901, 296, 14216, 13, 39492, 13, 47500, 13, 3387, 22029, 15], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]})

In [ ]:
tokenizer=AutoTokenizer.from_pretrained("allenai/OLMo-7B-0724-Instruct-hf")
tokenized = retain_train_df["input"].apply(
    tokenizer
)

In [ ]:
tokenized[1]

{'input_ids': [1276, 310, 253, 4201, 3522, 273, 26485, 66, 41131, 32], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
a

0       [input_ids, attention_mask]
1       [input_ids, attention_mask]
2       [input_ids, attention_mask]
3       [input_ids, attention_mask]
4       [input_ids, attention_mask]
                   ...             
1131    [input_ids, attention_mask]
1132    [input_ids, attention_mask]
1133    [input_ids, attention_mask]
1134    [input_ids, attention_mask]
1135    [input_ids, attention_mask]
Name: input, Length: 1136, dtype: object

Madera, a bustling city, served as the backdrop for the lives of four young women: Sherry, Margaretha, Lindsey, and Odilia. Their worlds intertwined through the seemingly mundane aspects of life, yet they remained strangers, unaware of the unbreakable bond that would soon unite them. Sherry, a quiet and imaginative soul, found solace in the city's art scene. Her paintings captured the vibrant colors and hidden stories of Madera, revealing a world that few could see. Margaretha, a brilliant student, pursued her passion for mathematics, solving complex equations and uncovering patterns that connected the world around her. Lindsey, a charismatic leader, worked tirelessly to unite the city's diverse communities, ensuring that everyone had a voice. Odilia, a compassionate soul, dedicated her time to volunteering at the local animal shelter, caring for the abandoned and neglected creatures that found their way to her.
As fate would have it, their paths crossed one day in the heart of Madera.

In [ ]:
forget_set = retain_train_df.dropna()


In [ ]:
len(forget_set)

1136

In [ ]:
a=0
for x,y in zip(retain_validation_df["input"],forget_validation_df["input"]):
    print(f"Forget:{y}")
    print(f"Retain{x}")
    a+=1
    

Forget:Madera, a bustling city, served as the backdrop for the lives of four young women: Sherry, Margaretha, Lindsey, and Odilia. Their worlds intertwined through the seemingly mundane aspects of life, yet they remained strangers, unaware of the unbreakable bond that would soon unite them. Sherry, a quiet and imaginative soul, found solace in the city's art scene. Her paintings captured the vibrant colors and hidden stories of Madera, revealing a world that few could see. Margaretha, a brilliant student, pursued her passion for mathematics, solving complex equations and uncovering patterns that connected the world around her. Lindsey, a charismatic leader, worked tirelessly to unite the city's diverse communities, ensuring that everyone had a voice. Odilia, a compassionate soul, dedicated her time to volunteering at the local animal shelter, caring for the abandoned and neglected creatures that found their way to her.
RetainChristopher J. Hardy

Christopher J. Hardy (born 1955) is an 

In [ ]:
a

254

In [ ]:
len(retain_train_df)

1136

In [ ]:
len(forget_train_df)

1112

In [ ]:
len(retain_validation_df)

278

In [ ]:
len(forget_validation_df)

254